In [ ]:
using Gridap        
using GridapGmsh
using Gridap.Geometry
using Gridap.TensorValues
using Plots
using LinearAlgebra
using  Gridap.Fields
using  Gridap.CellData
using  Gridap.ReferenceFEs  
using  Gridap.Fields
using Random
using LinearAlgebra
using Gridap.Arrays
using Gridap.CellData
using Base.Iterators

### Note: User need to create a .msh file before running the code

In [ ]:
model = GmshDiscreteModel("ThreePointBending.msh")
writevtk(model,"ThreePointBending")

In [ ]:
const E = 20000 
const ν = 0.2 
const G = E/(2*(1+ν))
const λ_ps = (E*ν)/((1+ν)*(1-2*ν)) # plane strain
const μ = G
const λ = λ_ps*(2*μ/(λ_ps+2*μ)) # plane stress
const G0 = 0.145
const fₜ = 2.4
const κ = λ + μ
r = 3.0  # (need to change based on input mesh)

In [ ]:
labels = get_face_labeling(model)

In [ ]:
order = 1
reffe = ReferenceFE(lagrangian,VectorValue{2,Float64},order)
V = TestFESpace(model,reffe;
          conformity=:H1,
          dirichlet_tags=["LeftEdge","RightEdge","LoadEdge"],
          dirichlet_masks=[(true,true),(false,true),(false,true)])
uh = zero(V)

In [ ]:
order = 1
reffeG = ReferenceFE(lagrangian,VectorValue{1,Float64},order)
VG = TestFESpace(model,reffeG;
          conformity=:H1)

order = 1
reffephi  = ReferenceFE(lagrangian,Float64,order)
Vphi  = TestFESpace(model,reffephi;
          conformity=:H1)
f_new = 1.0

In [ ]:
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)

In [ ]:
LoadTagId = get_tag_from_name(labels,"LoadEdge")
Γ_Load = BoundaryTriangulation(model,tags = LoadTagId)
dΓ_Load = Measure(Γ_Load,degree)
n_Γ_Load = get_normal_vector(Γ_Load)

In [ ]:
Gr = get_grid(model)
nodes = get_node_coordinates(Gr)
Nₑ, Nₙ = num_cells(model), num_nodes(model)
nodeCoordX, nodeCoordY = [nodes[i][1] for i in 1:Nₙ], [nodes[i][2] for i in 1:Nₙ]
elem = get_cell_node_ids(Gr)

In [ ]:
function nonlocalfield(Gk_nds, t)
    ϕVec_st = similar(Gk_nds)  
    @. ϕVec_st = exp(-(Gk_nds * t))  
    return FEFunction(Vphi, ϕVec_st)
end

In [ ]:
using NearestNeighbors
data = zeros(2,Nₙ)
data[1,:] =nodeCoordX
data[2,:] =nodeCoordY

points = data

balltree = BallTree(data)
idxs = inrange(balltree, points, r, true)

In [ ]:
function nonLocalGk(G_k_prev,t_k_prev)
    GkVec = evaluate(G_k_prev,x_S)
    TkVec = evaluate(t_k_prev,x_S)

    caches = [array_cache(GkVec) for k in 1:Threads.nthreads()]
    caches_T = [array_cache(TkVec) for k in 1:Threads.nthreads()]

    
    Gk_nds = zeros(Nₙ)
    Tk_nds = zeros(Nₙ)

    Threads.@threads for iel in 1:Nₑ
        cache = caches[Threads.threadid()]
        cache_T = caches_T[Threads.threadid()]
        ElNdInd = elem[iel]
        @inbounds Gk_nds[ElNdInd] = (getindex!(cache,GkVec,iel)).*(ones(3))
        @inbounds Tk_nds[ElNdInd] = (getindex!(cache_T,TkVec,iel)).*(ones(3))
    end

    Gk_nds_NL = zeros(Nₙ)
    Threads.@threads for nd_id in 1:Nₙ
        NeighHood = idxs[nd_id]
        @inbounds Gk_nds_NL[nd_id] = sum(Gk_nds[NeighHood]) / length(NeighHood)
    end

    return Gk_nds_NL,Tk_nds
end

In [ ]:
px = get_physical_coordinate(Ω)
Qₕ = CellQuadrature(Ω,1)
Qₕ_cell_point = get_cell_points(Qₕ)
dΩ_ro = Measure(Ω,1)
x_S = get_cell_points(dΩ_ro)

In [ ]:
σ(ε_nl) =  λ*tr(ε_nl)*one(ε_nl) + 2*μ*ε_nl

In [ ]:
function G_kill(σ_eq,dot_σ_eq)
 G_kill = 0.25*G0.*(( (σ_eq./fₜ -1)+ abs∘(σ_eq./fₜ - 1))).*((dot_σ_eq)+ abs∘(dot_σ_eq))
    return G_kill
end

In [ ]:
function new_EnergyState(ψPlusPrev_in,t_s_in,ψhPos_in)
    ψPlus_in = ψhPos_in
    if ψPlus_in <= ψPlusPrev_in
        ψPlus_out = ψPlusPrev_in 
         tPlus_out = t_s_in
        damaged = false
    else
        ψPlus_out = ψPlus_in
        tPlus_out = T
        damaged = true
    end
    damaged, ψPlus_out, tPlus_out   
end

In [ ]:
function project(q,model,dΩ,order)
  reffe = ReferenceFE(lagrangian,Float64,order)
  V = FESpace(model,reffe,conformity=:L2)
  a(u,v) = ∫( u*v )*dΩ
  l(v) = ∫( v*q )*dΩ
  op = AffineFEOperator(a,l,V,V)
  qh = solve(op)
  qh
end

In [ ]:
function σ_eq(ε, ε_nl)
    εArray, εArray_nl = get_array.((ε, ε_nl))
    Λ, P = eigen(εArray)
    Λ_nl, P_nl = eigen(εArray_nl)

    
    Λpos = diagm(0 => max.(0, Λ))
    Λpos_nl = diagm(0 => max.(0, Λ_nl))

    
    εPos = TensorValue(P * Λpos * P')
    εPos_nl = TensorValue(P_nl * Λpos_nl * P_nl')

    
    ψPos = 0.5 * ((tr(ε) >= 0) * (λ * tr(ε) * tr(ε_nl)) + 2μ * (εPos ⊙ εPos_nl))

    return √(2 * ψPos * E)
end

In [ ]:
function step_disp(uApp,dot_σ_eq,t_cell,uh,G_k_cell,ϕ)
uApp1(x) = VectorValue(0.0,0.0)
uApp2(x) = VectorValue(0.0,0.0) 
uApp3(x) = VectorValue(0.0,-uApp) 
U = TrialFESpace(V,[uApp1,uApp2,uApp3])
σ_eq_s = σ_eq∘((ε(uh)),(ε(uh))*ϕ)
G_k_in = G_kill(σ_eq_s,dot_σ_eq)
update_state!(new_EnergyState,G_k_cell,t_cell,G_k_in)
G_k_nd_nl,t_nds = nonLocalGk(G_k_cell,t_cell)
ϕ = (nonlocalfield(G_k_nd_nl,t_nds)) 
a(u,v) = ∫( ε(v) ⊙ (σ∘((ε(u))*(ϕ+1e-6)) ))*dΩ
l(v) = 0.0
op = AffineFEOperator(a,l,U,V)
ls = LUSolver()
solver = LinearFESolver(ls)
uh = solve(solver,op)
    return uh, ϕ, G_k_cell, t_cell, G_k_nd_nl, t_nds 
end

In [ ]:
cd("3PBend")
cd("ExactIntegration")
cd("o15")
cd("3PBendMesh1p5")

In [ ]:
Tmax = 0.5
delT = 8e-3
vApp = 1.0
count_n = 0
T = 0.0
Load = Float64[]
Displacement = Float64[]
push!(Load, 0.0)
push!(Displacement, 0.0)
G_k_cell = CellState(0.0,dΩ_ro)
t_cell = CellState(T,dΩ_ro)
uh_prev = zero(V)
uh_in_FE = uh
ϕ_prev = interpolate_everywhere(f_new,Vphi)
ϕ = interpolate_everywhere(f_new,Vphi)
innerMax = 100
dot_σ_eq = (σ_eq∘(ε(uh),(ε(uh))) - σ_eq∘(ε(uh_in_FE),(ε(uh_in_FE))))./delT
σ_eq_s = σ_eq∘(ε(uh),(ε(uh)))
G_k_nd_nl,t_nds = nonLocalGk(G_k_cell,t_cell)
ϕ = (nonlocalfield(G_k_nd_nl,t_nds))
start_time = time()
while T <= Tmax
    count_n = count_n + 1
T = T + delT
uApp  = T*vApp
    print("\n Entering displacemtent step :", float(uApp))
 for inner = 1:innerMax
uh,ϕ,G_k_cell,t_cell,G_k_nd_nl,t_nds=  step_disp(uApp,dot_σ_eq,t_cell,uh,G_k_cell,ϕ)
e = uh - uh_in_FE
dot_σ_eq = (σ_eq∘(ε(uh),(ε(uh))*(ϕ+1e-6)))./T
err = sqrt(sum( ∫( e⊙e )*dΩ ))
ϕ_prev = ϕ
uh_in_FE = uh
print("\n error = ",float(err))
        if err < 1e-8
            break 
        end  
    end
Node_Force = sum(∫( n_Γ_Load ⋅ (σ∘( (ε(uh))*ϕ) )) *dΓ_Load)
push!(Load, abs(Node_Force[2]))
push!(Displacement, uApp)

tVec_st_proj = (FEFunction(Vphi,t_nds))  
G_k = (FEFunction(Vphi,G_k_nd_nl))
    if mod(count_n,5) == 0
writevtk(Ω,"results_NonLocal_$count_n",cellfields=["disp"=>uh,"phi"=>ϕ,"sig"=>σ_eq∘(ε(uh),(ε(uh))*ϕ)])   
    end
    end
end_time = time()
elapsed_time = end_time - start_time

In [ ]:
plot(Displacement,Load,legend=:topleft)

In [ ]:
using DelimitedFiles
Disp = writedlm("3PB_h1p5_CE_8e-3_mod.csv",  [Displacement Load], ',')